In [ ]:
"""
  Sofía Almeida Bruno
  https://archive.ics.uci.edu/ml/datasets/optical+recognition+of+handwritten+digits
"""

In [2]:
import numpy as np

In [3]:
# Leemos los datos de los ficheros
train = np.genfromtxt('./datos/optdigits.tra',  delimiter = ",")
test = np.genfromtxt('./datos/optdigits.tes',  delimiter = ",")